## generate narrative from resource

### import modules

In [50]:
from fhir.resources import construct_fhir_element
from json import dumps, loads
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date, timezone, timedelta
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from commonmark import commonmark
from htmlmin import minify

### Define variables

#### get CapStatement as json

In [51]:
fname = input('enter the CapStatment file name:')

enter the CapStatment file name: capabilitystatement-data-consumer-client.json


In [52]:
path = Path.cwd() / 'cs_source_file'/ fname
cs_py = construct_fhir_element('CapabilityStatement',path.read_text())
print(cs_py.id)

data-consumer-client


### Create Profile url:Name mappings

In [53]:
# this is wrong if for most IGs should be supportedProfiles not Profiles
pname_map = {r.profile:titlecase(r.profile.split('/')[-1]) for r in cs_py.rest[0].resource if r.profile}
#add in supported Profiles too
spname_map = {sp:titlecase(sp.split('/')[-1]) for \
              r in cs_py.rest[0].resource if r.supportedProfile for sp in r.supportedProfile}
#flatten_matrix = [val for sublist in matrix for val in sublist] 

# Adding elements from dict2 to dict1
pname_map.update(spname_map)
pname_map

{'http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-claim': 'Profile Claim',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request': 'Cdex Task Data Request'}

### get pathmap from package file
- Get spec_internal from package a json file which includes canonical to local relative page links

In [54]:
ig_package = input("Get package path to download:")

Get package path to download: http://build.fhir.org/ig/HL7/davinci-ecdx/package.tgz


In [55]:
r = get(ig_package, verify=False)
r.status_code
#write to file
path = Path.cwd() /  'cs_source_file' / ig_package.split('/')[-1]
path.write_bytes(r.content)

261656

In [56]:
import tarfile

def get_si(tar_file):
    print(path)
    with tarfile.open(tar_file, mode='r') as tf:
        pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return(loads(r))


si = get_si(path)
       
path_map = si['paths']


#can = 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-inpatient-facility'
#path_map[can]
path_map

/Users/ehaas/Documents/Python/MyNotebooks/CapStatement/cs_source_file/package.tgz
['package/package.json',
 'package/ig-r4.json',
 'package/ImplementationGuide-hl7.fhir.us.davinci-cdex.json',
 'package/StructureDefinition-cdex-task-data-request.json',
 'package/CapabilityStatement-data-consumer-server.json',
 'package/ValueSet-cdex-attachment-reason.json',
 'package/CapabilityStatement-data-consumer-client.json',
 'package/CapabilityStatement-data-source-server.json',
 'package/CodeSystem-cdex-temp.json',
 'package/ValueSet-cdex-POU.json',
 'package/OperationDefinition-submit-attachment.json',
 'package/CapabilityStatement-data-source-client.json',
 'package/ValueSet-cdex-work-queue.json',
 'package/example/Task-cdex-example1-authorized-request.json',
 'package/example/Bundle-cdex-document-digital-sig-example.json',
 'package/example/Task-cdex-example1-query-completed-contained.json',
 'package/example/Task-cdex-example1s-query-requested-withsig.json',
 'package/example/Bundle-cdex-ele

{'http://hl7.org/fhir/us/davinci-cdex/ImplementationGuide/hl7.fhir.us.davinci-cdex|1.0.0': '1.0.0/ImplementationGuide-hl7.fhir.us.davinci-cdex.html',
 'http://hl7.org/fhir/us/davinci-cdex/ImplementationGuide/hl7.fhir.us.davinci-cdex': 'ImplementationGuide-hl7.fhir.us.davinci-cdex.html',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request|1.0.0': '1.0.0/StructureDefinition-cdex-task-data-request.html',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request': 'StructureDefinition-cdex-task-data-request.html',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-server|1.0.0': '1.0.0/CapabilityStatement-data-consumer-server.html',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-server': 'CapabilityStatement-data-consumer-server.html',
 'http://hl7.org/fhir/us/davinci-cdex/ValueSet/cdex-attachment-reason|1.0.0': '1.0.0/ValueSet-cdex-attachment-reason.html',
 'http://hl7.org/fhir/us/davinci-cde

### Get URL:Name/Title mapping from package file

In [57]:
from json import JSONDecodeError
def get_name_map(tar_file):
    print(path)
    name_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            f = tf.extractfile(p_file)
            r = f.read()
            #print(type(r))
            try:
                r_dict = loads(r)
            except JSONDecodeError: # not a json file
                continue
            try:
                name_map[r_dict['url']] = r_dict['title']
                #print(r_dict['title'])
            except KeyError:
                try:
                    name_map[r_dict['url']] = r_dict['name']
                    #print(r_dict['name'])
                except KeyError: # not a conf resource
                    print(f'****{p_file}is not a conf resource***')
        return(name_map)


name_map = get_name_map(path)
name_map       

/Users/ehaas/Documents/Python/MyNotebooks/CapStatement/cs_source_file/package.tgz
****package/example/Task-cdex-example1-authorized-request.jsonis not a conf resource***
****package/example/Bundle-cdex-document-digital-sig-example.jsonis not a conf resource***
****package/example/Task-cdex-example1-query-completed-contained.jsonis not a conf resource***
****package/example/Task-cdex-example1s-query-requested-withsig.jsonis not a conf resource***
****package/example/Bundle-cdex-electronic-sig-example.jsonis not a conf resource***
****package/example/Task-cdex-example3-query-requested.jsonis not a conf resource***
****package/example/Task-cdex-example3-query-completed.jsonis not a conf resource***
****package/example/Task-cdex-example1s-query-withsig-completed.jsonis not a conf resource***
****package/example/Task-cdex-example2-query-requested.jsonis not a conf resource***
****package/example/Task-cdex-example2-query-failed.jsonis not a conf resource***
****package/example/Bundle-cdex-se

{'https://build.fhir.org/ig/HL7/davinci-ecdx/': 'Da Vinci Clinical Data Exchange (CDex) Implementation Guide',
 'http://hl7.org/fhir/us/davinci-cdex/ImplementationGuide/hl7.fhir.us.davinci-cdex': 'Da Vinci Clinical Data Exchange (CDex) Implementation Guide',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request': 'CDex Task Data Request Profile',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-server': 'Data Consumer Server CapabilityStatement',
 'http://hl7.org/fhir/us/davinci-cdex/ValueSet/cdex-attachment-reason': 'CDex Attachment Reason Value Set',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-client': 'Data Consumer Client CapabilityStatement',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-source-server': 'Data Source Server CapabilityStatement',
 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp': 'CDex Temporary Code System',
 'http://hl7.org/fhir/us/davinci-cdex/ValueSet/cdex-PO

### Get SearchParameter parameter name: type mapping from package file

In [58]:
def get_sp_map(tar_file):
    print(path)
    sp_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            if 'searchparameter' in p_file.lower():
                print(f'**********{p_file}')
                f = tf.extractfile(p_file)
                r = f.read()
                #print(type(r))
                try:
                    r_dict = loads(r)
                except JSONDecodeError: # not a json file
                    continue
                try:
                    sp_map[r_dict['code']] = r_dict['type']
                    print(r_dict['type'])
                except KeyError:
                    print(f'{p_file} has no type')
        return(sp_map)


sp_map = get_sp_map(path)
sp_map       

/Users/ehaas/Documents/Python/MyNotebooks/CapStatement/cs_source_file/package.tgz


{}

### render using Jinja2 templates

create all maps:
- purl_map (canon:url)
- cs_map just canon:canon for now 
- ig_map just canon:canon for now 

In [59]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'],),
    trim_blocks = True,
    lstrip_blocks = True,
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

purl_map={} # canon:url, 
for p,n in pname_map.items():
    try:
        purl_map[p] = pname_map[p]
    except KeyError:
        purl_map[p] = p
try:
    cs_list = cs_py.instantiates + cs_py.imports
except TypeError:
    cs_list =[]
cs_map = {k:k for k in cs_list} # just canon:canon for now 
#-TODO see how can look up fhir registry and populate
ig_map = {k:k for k in cs_py.implementationGuide}

rendered = template.render(cs=cs_py, path_map=path_map, pname_map=name_map, purl_map=purl_map, sp_map=sp_map, 
                          csname_map=cs_map, csurl_map=cs_map, igname_map=ig_map, igurl_map=ig_map)

### Minify the xhtml

In [60]:
def x_minify(xhtml):
    h_min=minify(xhtml, remove_optional_attribute_quotes=False, remove_comments=True)
    x_min = h_min.replace('<br>','<br />')
    x_min = x_min.replace('<hr>','<hr />')
    return x_min

mini = x_minify(rendered)
#print(type(mini))
#print(mini)
#display(HTML(rendered))
display(HTML(mini))

'''
#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative_pre.xhtml'
path.write_text(rendered, encoding="utf-8")
path = Path.cwd() / 'debug' / 'narrative_mini_pre.xhtml'
path.write_text(mini, encoding="utf-8")
#===================================================
'''
narr = construct_fhir_element('Narrative',dict(
    status = 'generated',
    div = mini,
 ))
cs_py.text = narr

Resource Type,Supported Profiles,Supported Searches,Supported _includes,Supported _revincludes,Supported Operations
Bundle,-,-,-,-,-
Claim,,-,-,-,-
Subscription,-,-,-,-,-
Task,CDex Task Data Request Profile,-,-,-,-


### Validation

In [61]:

def validate(r):
    #fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r3'
    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    fhir_test_server ='http://hapi.fhir.org/baseR4'
  

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    r = post(f'{fhir_test_server}/CapabilityStatement/$validate', headers = headers, data = cs_py.json())   # return r.status_code
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

validate(cs_py)

ERROR,[]


### Save to local dir

In [62]:
# save to file
ig_source_path = ''
print('...........saving to file............')
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs_py.id.lower()}.json'
path.write_text(cs_py.json(indent=2))

...........saving to file............


25138